In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

# Spark Session initialization
spark = SparkSession.builder.appName("Example").getOrCreate()


24/03/20 23:02:14 WARN Utils: Your hostname, yuzhuorandeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.4.42 instead (on interface en0)
24/03/20 23:02:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/20 23:02:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/20 23:02:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)

### select 100 stocks from Shanghai Stock Exchange(SSE)

In [4]:
dim_stock_df = spark.read.csv('Dim_Stock.csv', header=True, inferSchema=True)
sse_stocks_df = dim_stock_df.filter(col("stock_id") > 600000).orderBy("stock_id").limit(10)
sse_stocks_df.select('Stock_ID', 'Total_Value').show()

+--------+--------------------+
|Stock_ID|         Total_Value|
+--------+--------------------+
|  600004|   2.582089646753E10|
|  600006|            1.282E10|
|  600007|    2.11025690873E10|
|  600008|2.011321845498000...|
|  600009|9.135214999140001E10|
|  600010|   7.173980875184E10|
|  600011|   1.161658908566E11|
|  600012|       1.71500274E10|
|  600015|    9.07150922676E10|
|  600016|  1.6768666286266E11|
+--------+--------------------+



### --select all stocks that rise over 9.9% on 2024-03-11

In [5]:
fact_stock_daily_df = spark.read.csv('Fact_Stock_Daily_2024.csv', header=True, inferSchema=True)
rising_stocks_df = fact_stock_daily_df.filter((col("Date_ID") == 20240311) & (col("percent_of_incre_decre") > 9.9))\
                    .orderBy(col("percent_of_incre_decre").desc())
rising_stocks_df.show()

+--------------+--------+--------+-----+------+-----+-------+---------------+---------+----------------------+---------------------+-------------+------+
|        FSD_ID|Stock_ID| Date_ID| Open|  High|  Low| Volume|       Turnover|Amplitude|Percent_Of_Incre_Decre|Amount_Of_Incre_Decre|Turnover_Rate| Close|
+--------------+--------+--------+-----+------+-----+-------+---------------+---------+----------------------+---------------------+-------------+------+
|20240311300317|  300317|20240311| 4.04|  4.84| 4.04| 972902|   4.60648926E8|    19.85|                  20.1|                 0.81|        11.76|  4.84|
|20240311300324|  300324|20240311| 2.98|  3.29| 2.88|1842843|   5.68963029E8|    14.96|                 20.07|                 0.55|        10.79|  3.29|
|20240311300530|  300530|20240311| 17.8|  21.4| 17.8|  94640|   1.91813994E8|    20.19|                 20.02|                 3.57|         5.98|  21.4|
|20240311301292|  301292|20240311|14.16| 16.85|14.16|  50492|    8.4292834E7

### select information from 20231120 9:30-9:40 on stock SH600004

In [10]:
import time
start_time=  time.time()
fact_stock_minute_df = spark.read.csv('Fact_Stock_Minute.csv', header=True, inferSchema=True)
stock_600004_info_df = fact_stock_minute_df.filter((col("Minute_ID") > 202311200930) \
                                                   & (col("Minute_ID") < 202311200940) \
                                                    & (col("Stock_ID") == 600004))
stock_600004_info_df.show()
print(f"Time used: {time.time() - start_time}")

+------------------+--------+------------+----+-----+-----+------+----------+---------+-----+
|            FSM_ID|Stock_ID|   Minute_ID|Open| High|  Low|Volume|Stock_Name| Turnover|Close|
+------------------+--------+------------+----+-----+-----+------+----------+---------+-----+
|600004202311200931|  600004|202311200931| 0.0|10.92|10.89|   647|   ���ƻ���| 705561.0|10.91|
|600004202311200932|  600004|202311200932| 0.0| 10.9|10.89|   401|   ���ƻ���| 436909.0|10.89|
|600004202311200933|  600004|202311200933| 0.0|10.91|10.86|  2568|   ���ƻ���|2794153.0| 10.9|
|600004202311200934|  600004|202311200934| 0.0|10.91| 10.9|   310|   ���ƻ���| 338083.0| 10.9|
|600004202311200935|  600004|202311200935| 0.0|10.95|10.91|  1727|   ���ƻ���|1886825.0|10.94|
|600004202311200936|  600004|202311200936| 0.0|10.94|10.92|   202|   ���ƻ���| 220870.0|10.93|
|600004202311200937|  600004|202311200937| 0.0|10.94|10.93|  1230|   ���ƻ���|1345150.0|10.93|
|600004202311200938|  600004|202311200938| 0.0|10.93|10.93| 

### show total minutes in 2024-03-11

In [9]:
dim_minute_df = spark.read.csv('Dim_Minute_2024.csv', header=True, inferSchema=True)
total_minutes_df = dim_minute_df.filter(col("Date").cast("date") == "2024-03-11")\
                                .agg({"Minute_ID": "count"})\
                                .withColumnRenamed("count(Minute_ID)", "Total_Minutes")
total_minutes_df.show()

+-------------+
|Total_Minutes|
+-------------+
|          241|
+-------------+



### show total trading days in Feburary

In [11]:
dim_date_df = spark.read.csv('Dim_Date_2024.csv', header=True, inferSchema=True)
total_trading_days_df = dim_date_df.filter((col("year") == 2024) & (col("month") == 2))\
    .agg({"Date_ID": "count"})\
    .withColumnRenamed("count(Date_ID)", "Total_Trading_Days")
total_trading_days_df.show()

+------------------+
|Total_Trading_Days|
+------------------+
|                16|
+------------------+



### show recent company events from September (company code between 600000 and 600010)

In [27]:
fact_company_event_df = spark.read.csv('Fact_Company_Event.csv', header=True, inferSchema=True).withColumnRenamed("Stock_ID","FCE_Stock_ID")
dim_event_df = spark.read.csv('Dim_Event.csv', header=True, inferSchema=True).withColumnRenamed("Stock_ID","DE_Stock_ID")

recent_events_df = fact_company_event_df.join(dim_event_df, fact_company_event_df.Event_Dim_ID == dim_event_df.Event_Dim_ID, "inner")\
                    .select("FCE_ID","FCE_Stock_ID","Date_ID","Event_Type","Current_Flag","Effective_Timestamp","Expire_Timestamp")\
                    .filter((col("Date_ID") > 20230901) & (col("FCE_Stock_ID") > 600000) & (col("FCE_Stock_ID") < 600010))
                    
recent_events_df.show()

+------------------+------------+--------+-------------------+------------+-------------------+----------------+
|            FCE_ID|FCE_Stock_ID| Date_ID|         Event_Type|Current_Flag|Effective_Timestamp|Expire_Timestamp|
+------------------+------------+--------+-------------------+------------+-------------------+----------------+
|202309150082600006|      600006|20230915|Asset Restructuring|           1|         2023-09-15|            NULL|
|202311140063600008|      600008|20231114|Asset Restructuring|           1|         2023-11-14|            NULL|
+------------------+------------+--------+-------------------+------------+-------------------+----------------+



### select Minute Level Data from 2023-11-21 9:30 to 2023-11-21 9:40 where the company is in CN market and its Total_Value is in top 10

In [30]:
start_time=  time.time()
top_10_cn_stocks_cached = dim_stock_df.filter(col("Stock_Market") == "CN") \
    .orderBy(col("Total_Value").desc()) \
    .limit(10) \
    .select("Stock_ID") \
    .withColumnRenamed("Stock_ID","top10_Stock_ID")\
    .cache()


minute_level_data_df = fact_stock_minute_df.join(top_10_cn_stocks_cached, \
                        fact_stock_minute_df.Stock_ID == top_10_cn_stocks_cached.top10_Stock_ID, "inner") \
    .filter((col("Minute_ID") >= 202311210930) & (col("Minute_ID") <= 202311210940)) \
    .select("FSM_ID", "Stock_ID", "Minute_ID", "Close", "High", "Low", "volume")

minute_level_data_df.show()

top_10_cn_stocks_cached.unpersist()
print(f"Time used: {time.time() - start_time}")

+------------------+--------+------------+-----+----+----+------+
|            FSM_ID|Stock_ID|   Minute_ID|Close|High| Low|volume|
+------------------+--------+------------+-----+----+----+------+
|601288202311210930|  601288|202311210930| 3.66|3.66|3.66| 11861|
|601288202311210931|  601288|202311210931| 3.66|3.67|3.65|155799|
|601288202311210932|  601288|202311210932| 3.66|3.67|3.65|116492|
|601288202311210933|  601288|202311210933| 3.67|3.67|3.66| 11392|
|601288202311210934|  601288|202311210934| 3.66|3.67|3.66| 14070|
|601288202311210935|  601288|202311210935| 3.67|3.67|3.66| 28420|
|601288202311210936|  601288|202311210936| 3.66|3.67|3.66| 14610|
|601288202311210937|  601288|202311210937| 3.67|3.67|3.66|  9740|
|601288202311210938|  601288|202311210938| 3.66|3.67|3.66|  9401|
|601288202311210939|  601288|202311210939| 3.66|3.67|3.66| 10829|
|601288202311210940|  601288|202311210940| 3.67|3.67|3.66| 38237|
|601857202311210930|  601857|202311210930| 7.14|7.14|7.14|  4878|
|601857202

In [7]:
spark.stop()